<a href="https://colab.research.google.com/github/JSJeong-me/Local-LLM/blob/main/05-gemma_7b-it.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip -q install git+https://github.com/huggingface/transformers # need to install from github
!pip install -q datasets loralib sentencepiece
!pip -q install bitsandbytes accelerate xformers einops
!pip -q install hf_transfer

https://huggingface.co/google/gemma-7b-it

In [3]:
!nvidia-smi

Fri Mar  8 08:07:25 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [4]:
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import os


#os.environ ['HF_HUB_ENABLE_HF_TRANSFER'] = '1'

In [5]:
torch.set_default_device('cuda')

In [ ]:
quantization_config = BitsAndBytesConfig(load_in_4bit=True)

model = AutoModelForCausalLM.from_pretrained("google/gemma-7b-it", # 'google/gemma-2b-it
                                             quantization_config=quantization_config,
                                             low_cpu_mem_usage=True,
                                             torch_dtype="auto",
                                             device_map="auto"
                                             )

tokenizer = AutoTokenizer.from_pretrained("google/gemma-7b-it")

config.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

In [ ]:
tokenizer.vocab_size

In [ ]:
chat = [
    { "role": "user", "content": "What is the difference between LLaMAs, Alpacas, and Vicunas" },
]

prompt = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)
prompt


In [ ]:
inputs = tokenizer.encode(prompt, add_special_tokens=True, return_tensors="pt")

outputs = model.generate(input_ids=inputs.to(model.device), max_new_tokens=512)

In [ ]:
from IPython.display import Markdown, display
display(Markdown("## Gemma - 7B 4Bit"))

In [ ]:
text = tokenizer.decode(outputs[0],skip_special_tokens=True, clean_up_tokenization_spaces=True)
# text = text.replace(prompt, '', 1)
display(Markdown(text))

In [ ]:
chat = [
    { "role": "user", "content": "What is the difference between LLaMAs, Alpacas, and Vicunas" },
]

prompt = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)
prompt


In [ ]:
prompt

### Prompt Format
```
<start_of_turn>user
What is the difference between LLaMAs, Alpacas, and Vicunas<end_of_turn>
<start_of_turn>model



```

In [ ]:
import textwrap

def wrap_text(text, width=90): #preserve_newlines
    # Split the input text into lines based on newline characters
    lines = text.split('\n')

    # Wrap each line individually
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]

    # Join the wrapped lines back together using newline characters
    wrapped_text = '\n'.join(wrapped_lines)

    return wrapped_text

In [ ]:
def generate(input_text, system_prompt="",max_length=512):
    if system_prompt != "":
        system_prompt = system_prompt
    else:
        system_prompt = "You are a friendly and helpful assistant"
    messages = [
        # {
        #     "role": "system",
        #     "content": system_prompt,
        # },
        {"role": "user", "content": system_prompt + '\n\n' +input_text},
    ]

    prompt = tokenizer.apply_chat_template(messages,
                                                tokenize=False,
                                                add_generation_prompt=True)

    inputs = tokenizer.encode(prompt, add_special_tokens=True, return_tensors="pt")
    outputs = model.generate(input_ids=inputs.to(model.device),
                             max_new_tokens=max_length,
                             do_sample=True,
                             temperature=0.1,
                             top_k=50,
                             )
    text = tokenizer.decode(outputs[0],skip_special_tokens=True, clean_up_tokenization_spaces=True)
    text = text.replace('user\n'+system_prompt+ '\n\n' +input_text+ '\nmodel', '', 1)
    wrapped_text = wrap_text(text)
    display(Markdown(wrapped_text))

## Instruction Answering

In [ ]:
generate('Write a detailed analogy between mathematics and a lighthouse.',
         system_prompt="You are Gemma, a large language model trained by Google. Write out your reasoning step-by-step to be sure you get the right answers!",
         max_length=512)

In [ ]:
%%time
generate('Write a detailed analogy between mathematics and a music.',
         system_prompt="You are Gemma, a large language model trained by Google. Write out your reasoning step-by-step to be sure you get the right answers!",
         max_length=256)

In [ ]:
%%time
generate('What is the difference between a Llama, Vicuna and an Alpaca?',
         system_prompt="Write out your reasoning step-by-step to be sure you get the right answers!",
         max_length=512)

In [ ]:
%%time
generate('Write a short email to Sam Altman giving reasons to open source GPT-4',
         system_prompt="You are Gemma, a large language model trained by Google. Write out your reasoning step-by-step to be sure you get the right answers!",
         max_length=512)

In [ ]:
%%time
generate('Write a short email to Sam Altman giving reasons to open source GPT-4',
         system_prompt="You are Freddy a young 5 year old boy who is scared AI will end the world, write only with the language of a young child!",
         max_length=512)

In [ ]:
%%time
generate('Write a short email to Sam Altman giving reasons to open source GPT-4',
         system_prompt="You are Kate, the Vice president of USA, you are against regulatory capture and like to explain that!",
         max_length=512)

In [ ]:
%%time
generate('What is the capital of England?',
         system_prompt="You are Gemma, a large language model trained by Google. Write out your short and succinct!",
         max_length=256)

## CodeGen

In [ ]:
generate('''```python
def print_prime(n):
   """
   Print all primes between 1 and n
   """''',
         system_prompt="You are a genius python coder, please think carefully and write the following code:")

## GSM8K

In [ ]:
generate('Answer the following question by reasoning step by step. The cafeteria had 23 apples. If they used 20 for lunch, and bought 6 more, how many apple do they have?',
         system_prompt="You are Gemma, a large language model trained by Google. Write out your reasoning step-by-step to be sure you get the right answers!",
         max_length=256)

In [ ]:
generate("x + 2x + 4x =  847 What is x?",
         system_prompt="You are Gemma, a large language model trained by Google. Write out your reasoning step-by-step to be sure you get the right answers!",
         max_length=2048)

In [ ]:
generate("How do I say 'How can I get to Orchard road?' in Singlish?",
         system_prompt="You are Gemma, a large language model trained by Google. Write out your reasoning step-by-step to be sure you get the right answers!",
         max_length=2048)

In [ ]:
generate("How do I say 'How can I get to Orchard road?' in Thai?",
         system_prompt="You are Gemma, a large language model trained by Google. Write out your reasoning step-by-step to be sure you get the right answers!",
         max_length=2048)

- Source: [Sam Witteveen](https://www.youtube.com/watch?v=0xhZ2OhGNDg)